## B. Setup & Package
### 1. Import library & penyeseuaian lingkungan notebook

Pada sel berikut saya impor library yang akan digunakan dan melakukan penyesuaian terhadap notebook saya.

In [26]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from sklearn.preprocessing import MinMaxScaler

### 2. Penyimpanan `PATH` file

In [7]:
raw_path = {
    'calendar.gz' : '../data/raw/calendar.csv.gz',
    'listings' : '../data/raw/listings.csv',
    'listings.gz' : '../data/raw/listings.csv.gz',
    'neighbourhoods' : '../data/raw/neighbourhoods.csv',
    'neighbourhoods.geo' : '../data/raw/neighbourhoods.geojson',
    'reviews' : '../data/raw/reviews.csv',
    'reviews.gz' : '../data/raw/reviews.csv.gz'
}

## C. Data Preparation
### 1. Baca dataset dari data yang ada sesuai **path**

In [25]:
calendar_gz = pd.read_csv(raw_path['calendar.gz'])

listings = pd.read_csv(raw_path['listings'])
listings_gz = pd.read_csv(raw_path['listings.gz'])

neighbourhoods = pd.read_csv(raw_path['neighbourhoods'])
neighbourhoods_geo = gpd.read_file(raw_path['neighbourhoods.geo'])

reviews = pd.read_csv(raw_path['reviews'])
reviews_gz = pd.read_csv(raw_path['reviews.gz'])

- Cek fragmen dataset yang duplikat!
    - `listings` dan `listings_gz`
    - `neighbourhoods` dan `neighbourhood_.geo`
    - `reviews` dan `reviews_gz`

In [ ]:
print("5 Kolom Pertama listings:", listings.head())
print("5 Kolom Pertama listings_gz:", listings_gz.head())

5 Kolom Pertama listings:        id                                              name  host_id  \
0   71609               Ensuite Room (Room 1 & 2) near EXPO   367042   
1   71896                   B&B  Room 1 near Airport & EXPO   367042   
2   71903                        Room 2-near Airport & EXPO   367042   
3  275343  10min walk to MRT & a Cozy Room with window! (1)  1439258   
4  275344             15 mins to Outram MRT Single Room (2)  1439258   

  host_name neighbourhood_group neighbourhood  latitude  longitude  \
0   Belinda         East Region      Tampines   1.34537  103.95887   
1   Belinda         East Region      Tampines   1.34754  103.95958   
2   Belinda         East Region      Tampines   1.34531  103.96100   
3       Kay      Central Region   Bukit Merah   1.29015  103.80814   
4       Kay      Central Region   Bukit Merah   1.28836  103.81144   

      room_type  price  minimum_nights  number_of_reviews last_review  \
0  Private room    NaN              92         

In [27]:
print("5 Kolom Pertama neighbourhoods:", neighbourhoods.head())
print("5 Kolom Pertama neighbourhoods:", neighbourhoods_geo.head())

5 Kolom Pertama neighbourhoods:   neighbourhood_group  neighbourhood
0      Central Region         Bishan
1      Central Region    Bukit Merah
2      Central Region    Bukit Timah
3      Central Region  Downtown Core
4      Central Region        Geylang
5 Kolom Pertama neighbourhoods:    neighbourhood neighbourhood_group  \
0      Pasir Ris         East Region   
1        Seletar   North-East Region   
2   Sungei Kadut        North Region   
3        Orchard      Central Region   
4  Downtown Core      Central Region   

                                            geometry  
0  MULTIPOLYGON (((103.95322 1.38202, 103.9535 1....  
1  MULTIPOLYGON (((103.88691 1.42649, 103.88812 1...  
2  MULTIPOLYGON (((103.7644 1.44345, 103.76443 1....  
3  MULTIPOLYGON (((103.84298 1.30001, 103.84294 1...  
4  MULTIPOLYGON (((103.85942 1.29978, 103.85954 1...  


In [28]:
print("5 Kolom Pertama reviews:", reviews.head())
print("5 Kolom Pertama reviews_gz:", reviews_gz.head())

5 Kolom Pertama reviews:    listing_id        date
0       71609  2011-12-19
1       71609  2012-07-17
2       71609  2012-09-01
3       71609  2012-09-04
4       71609  2013-01-02
5 Kolom Pertama reviews_gz:    listing_id       id        date  reviewer_id reviewer_name  \
0       71609   793880  2011-12-19      1456140           Max   
1       71609  1731810  2012-07-17      1804182         Jaiya   
2       71609  2162194  2012-09-01      3113461         Zahra   
3       71609  2190615  2012-09-04      1432123        Helmut   
4       71609  3221837  2013-01-02      2759938          Jack   

                                            comments  
0  The rooms were clean and tidy. Beds very comfo...  
1  Good space and quite an interesting home in a ...  
2  It was a comfortable place. Belinda was a kind...  
3  We are four mature age travellers and stayed f...  
4  Belinda is a great host! She helps you wheneve...  
